In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point

In [61]:
df_full = pd.read_csv("data/zomato_df_final_data.csv")

In [26]:
# haven't decided what to do with cuisine
# everyone has address, link, phone and title
# not sure why we have color and cuisine_color
# for regression 1 we wont use rating_text
# won't need lat and lng cause we will use subzone
df = df_full.drop(columns = ["address", "link", "phone", "title", "color", "cuisine_color", "rating_text", "lat", "lng"], axis = 1)

In [27]:
df.dtypes

cost             float64
cuisine           object
rating_number    float64
subzone           object
type              object
votes            float64
groupon             bool
cost_2           float64
dtype: object

In [28]:
# do some feature engineering
# change groupon bool to int
df.groupon = df.groupon.astype(int)

In [29]:
df.isna().sum()

cost              346
cuisine             0
rating_number    3316
subzone             0
type               48
votes            3316
groupon             0
cost_2            346
dtype: int64

# Using geopy Nominatim to check if I can get lat and lng values for an address from df["address"]

In [30]:
# # bringing address to see in helps
# df["address"] = df_full["address"]

# # check the addresses where lat is na
# temp = df[df.lat.isna()]
# temp.address

In [32]:
# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut

In [31]:
# i = 0
# for address in temp.address : 
#     print(f"{i}. address : {address}")
#     i += 1
#     try : 
#         geolocator = Nominatim(user_agent="address_to_ssc_code")
#         location = geolocator.geocode(address)
#         if location:
#             latitude, longitude = location.latitude, location.longitude
#             print(f"Latitude: {latitude}, Longitude: {longitude}")
#         else:
#             print("Address not found.")
#     except GeocoderTimedOut:
#         print("Geocoding service timed out. Address not found.")
#     except Exception as e:
#         print(f"An error occurred: Max retries exceeded")

# CHANGE OF PLAN
# timeout most of the time, checked until 59 found about 10 addresses

# Where lat is na, we can split address by "," and get the -1 of the resulting array

In [33]:
# missing_suburbs = temp.address.apply(lambda x : x.split(",")[-1].strip())
# missing_suburbs = [suburb.strip() for suburb in missing_suburbs]

In [34]:
# missing_suburbs.unique()

In [36]:
# df.loc[df['lat'].isna(), "address"] = df['address'].str.split(",").str[-1].str.strip()

# CHANGE OF PLAN
# Have about 52 uniqe missing_suburb from 192 rows
# which is good
# but most of them are sydney and cbd which are same
# does not get the actual subzone cause some of the addresses are like - 20 Campbell Street, Chinatown, Sydney
# got sydney instead of chinatown from the previous example
# Hopeless!

# If df["subzone"] has "," that means the subzone is not correct, we can split by "," and get the last part

In [37]:
# subzone is not suburb or at leats it's not correct for 1359 entries
# suburb name dont have comma
len(df[df.subzone.str.contains(",")])

1359

In [38]:
df['subzone'] = df['subzone'].apply(lambda x: x.split(',')[-1].strip() if ',' in x else x)

In [39]:
len(df_full.subzone.unique())

572

In [40]:
len(df.subzone.unique())

339

In [42]:
df.isna().sum()

cost              346
cuisine             0
rating_number    3316
subzone             0
type               48
votes            3316
groupon             0
cost_2            346
dtype: int64

# Converting subzone to numeric value for the regression

In [69]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['subzone_encoded'] = label_encoder.fit_transform(df['subzone'])
df = df.drop(columns = ["subzone"], axis = 1)

# Will drop rows that has missing rating_number, if i do mediano or mean or mode, the model will be biased

In [43]:
df.dropna(subset=['rating_number'], inplace=True)

In [45]:
df.isna().sum()

cost             103
cuisine            0
rating_number      0
subzone            0
type              21
votes              0
groupon            0
cost_2           103
dtype: int64

# Replacing na in "cost" and "cost_2" with mean cost of the restaurants in that suburb

In [47]:
df['cost'] = df.groupby('subzone')['cost'].transform(lambda x: x.fillna(x.mean()))
df['cost_2'] = df.groupby('subzone')['cost_2'].transform(lambda x: x.fillna(x.mean()))

In [50]:
# just verifying
df.loc[793]

cost                          51.441718
cuisine          ['Chinese', 'Sichuan']
rating_number                       3.8
subzone                       Chinatown
type                  ['Casual Dining']
votes                              40.0
groupon                               0
cost_2                         5.355255
Name: 793, dtype: object

# Dropping cuisine cause 135 one-hot-encoding is not reasonable, also rating is not so much related to cuisine

In [53]:
df = df.drop(columns = ["cuisine"])

In [55]:
df.isna().sum()

cost              0
rating_number     0
subzone           0
type             21
votes             0
groupon           0
cost_2            0
dtype: int64

# Fix up type and do one-hot-encoding

In [57]:
# filling na with most common type
most_frequent = df["type"].mode()[0]
df["type"].fillna(most_frequent, inplace=True)

In [58]:
import re
types = set()

#custom function to check all the values
def get_type(str):
    pattern = r',\s*' #removing comma(,) followed by immediate space
    cleaned_text = re.sub(pattern, '-', str)
    pattern2 = r'[\'\[\],]'
    cleaned_text = re.sub(pattern2, "", cleaned_text)
    temp = cleaned_text.split("-")
    for type in temp:
        types.add(type)

In [65]:
for value in df["type"]:
    get_type(value)
len(types)

# I HAD 14 TYPES BEFORE REMOVING RATING_NUMBER
# MISSING FOOD STALL

13

In [66]:
df_full.loc[df_full.rating_number.isna(), "type"].unique()

array(["['Casual Dining']", "['Café']", "['Bar', 'Fine Dining']",
       "['Fast Food']", "['Casual Dining', 'Bar']",
       "['Café', 'Casual Dining']", "['Dessert Parlour', 'Bakery']",
       "['Pub', 'Bar']", "['Casual Dining', 'Fast Food']",
       "['Fine Dining', 'Wine Bar']", "['Food Court']", "['Bar']",
       "['Casual Dining', 'Café']", "['Pub']", "['Fine Dining']",
       "['Dessert Parlour', 'Café']", "['Food Court', 'Café']",
       "['Bar', 'Casual Dining']", "['Bakery', 'Café']",
       "['Café', 'Bar']", "['Café', 'Dessert Parlour']",
       "['Fast Food', 'Food Court']", "['Café', 'Bakery']",
       "['Food Truck']", nan, "['Café', 'Wine Bar']",
       "['Club', 'Casual Dining']", "['Bakery']", "['Beverage Shop']",
       "['Fine Dining', 'Bar']", "['Café', 'Food Court']",
       "['Dessert Parlour']", "['Casual Dining', 'Food Stall']",
       "['Wine Bar']", "['Club']", "['Bar', 'Club']",
       "['Food Court', 'Casual Dining']", "['Pub', 'Casual Dining']",
       "['

In [67]:
df.type.unique()

array(["['Casual Dining']", "['Café']", "['Fine Dining']",
       "['Fine Dining', 'Bar']", "['Casual Dining', 'Dessert Parlour']",
       "['Fine Dining', 'Wine Bar']", "['Casual Dining', 'Wine Bar']",
       "['Casual Dining', 'Bar']", "['Fast Food']", "['Café', 'Bakery']",
       "['Casual Dining', 'Café']", "['Bar']",
       "['Bakery', 'Dessert Parlour']", "['Bar', 'Wine Bar']",
       "['Food Court']", "['Dessert Parlour', 'Café']",
       "['Bar', 'Casual Dining']", "['Bakery', 'Pub']",
       "['Café', 'Casual Dining']", "['Café', 'Bar']",
       "['Casual Dining', 'Pub']", "['Bakery', 'Café']", "['Food Truck']",
       "['Café', 'Dessert Parlour']", "['Dessert Parlour']", "['Pub']",
       "['Pub', 'Casual Dining']", "['Dessert Parlour', 'Bakery']",
       "['Café', 'Wine Bar']", "['Pub', 'Wine Bar']",
       "['Food Court', 'Casual Dining']", "['Wine Bar', 'Casual Dining']",
       "['Casual Dining', 'Food Court']", "['Club', 'Bar']",
       "['Casual Dining', 'Bakery']", "['

In [60]:
types

{'Bakery',
 'Bar',
 'Beverage Shop',
 'Café',
 'Casual Dining',
 'Club',
 'Dessert Parlour',
 'Fast Food',
 'Fine Dining',
 'Food Court',
 'Food Truck',
 'Pub',
 'Wine Bar'}

In [68]:
# one hot encoding
for dining_type in types:
    df[f"is_{dining_type}"] = df['type'].apply(lambda x: 1 if dining_type in x else 0)
df.drop('type', axis=1, inplace=True)

# Regression model 1

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# X,y
X = df.drop(columns=['rating_number'])  # Features
y = df['rating_number']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model_regression_1 = LinearRegression()
model_regression_1.fit(X_train, y_train)

# predictions
y_pred = model_regression_1.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.1469842497479085
R-squared: 0.2539999238616345


# Regression model 2, will need to apply standardization for gradient descent

In [71]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_regression_2 = SGDRegressor(loss="squared_error", max_iter=1000, random_state=0)
model_regression_2.fit(X_train_scaled, y_train)

y_pred = model_regression_2.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.1491078750935937
R-squared: 0.24322173046820894
